## Pengambilan Data 


In [1]:
import pandas as pd
import time
import tweepy
import numpy as np

def twitter_config():
    """
    Fungsi utilitas untuk mengkonfigurasi konsumsi file API Twitter dengan␣
    ,!kunci yang disediakan.
    """
    # Otentikasi dan akses menggunakan kunci:
    auth = tweepy.OAuthHandler("2fqCjSqwb2Thb9jgW8s1gXBq3",
    "4qEqrnvomA4S6zwSI3C27KZ8OHJshwK13gikJom64hfTrHdhAT")
    auth.set_access_token("1569924724393938945-kCcwLjcnELsFyzTk7fN4zCmpKsDnad",
    "0QOR6BDWt0Pwc16zGID27JObvzLzn7RW4vjeujMAqP0Fz")
    # Kembalikan akses ke API:
    api = tweepy.API(auth)
    try:
        api.verify_credentials()
        print("Authentication OK")
    except:
        print("Error during authentication")
    return api

# buat extractor object
extractor = twitter_config()


tweets = extractor.user_timeline(screen_name="JeromePolin", count=100)
print("Tweets terambil: {}.\n".format(len(tweets)))


print("5 tweet teratas:\n")
for tweet in tweets[:5]:
    print(tweet.text)
    print()
    


Authentication OK
Tweets terambil: 100.

5 tweet teratas:

@laaaanis wkkwkw tengkiuuu

Kamu yang salah, kok kamu yang marah

Hari ini ke Lotte World Korea, pake seragam sekolah Korea wkwkwk. Seruuu bgt! Tungguin vlog Korea Trip yahh! https://t.co/CmszdIGiSc

@jehianps @samsungID Thx ingfo

RT @kamubukanakuu: PERKARA JAMBU JD PANJANG



In [2]:
# Kita dapat membuat kerangka data sebagai berikut:
dataset = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['Tweets'])

# Kami membuat tampilan kerangka data:
display(dataset.head(10))

dataset.to_csv('Data Mentah Jerome.csv',sep=',')

,Tweets
0,@laaaanis wkkwkw tengkiuuu
1,"Kamu yang salah, kok kamu yang marah"
2,"Hari ini ke Lotte World Korea, pake seragam se..."
3,@jehianps @samsungID Thx ingfo
4,RT @kamubukanakuu: PERKARA JAMBU JD PANJANG
5,RT @barkhohum: konspirasi
6,RT @yxxxb_07: Masuk akal sih 🤣
7,RT @sminyeop: diluar nalar wkwkkw
8,Viral seorang anak ditanya √25 = …? Dan anak t...
9,"Bentar lagi bakal trip ke Korea, gak sabar ket..."


## Pengolahan Data Set

In [3]:
import re
import string
import time
from copy import deepcopy
import pandas as pd

## Membaca Data Set

In [4]:
df = pd.read_csv('Data Mentah Jerome.csv')
df.head()

,Unnamed: 0,Tweets
0,0,@laaaanis wkkwkw tengkiuuu
1,1,"Kamu yang salah, kok kamu yang marah"
2,2,"Hari ini ke Lotte World Korea, pake seragam se..."
3,3,@jehianps @samsungID Thx ingfo
4,4,RT @kamubukanakuu: PERKARA JAMBU JD PANJANG


## Instalasi Package Ekpharasis

In [12]:
!pip install ekphrasis

In [5]:
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons

text_processor = TextPreProcessor(
    # terms that will be normalized
    normalize=['email', 'percent', 'money', 'phone', 'user',
        'time', 'date', 'number'],
    # terms that will be annotated
    #annotate={"hashtag", "allcaps", "elongated", "repeated",'emphasis', 'censored'},
    annotate={"hashtag"},
    fix_html=True,  # fix HTML tokens
    
    # corpus from which the word statistics are going to be used 
    # for word segmentation 
    segmenter="twitter", 
    
    # corpus from which the word statistics are going to be used 
    # for spell correction
    corrector="twitter", 
    
    unpack_hashtags=True,  # perform word segmentation on hashtags
    unpack_contractions=True,  # Unpack contractions (can't -> can not)
    spell_correct_elong=False,  # spell correction for elongated words
    
    # select a tokenizer. You can use SocialTokenizer, or pass your own
    # the tokenizer, should take as input a string and return a list of tokens
    tokenizer=SocialTokenizer(lowercase=True).tokenize,
    
    # list of dictionaries, for replacing tokens extracted from the text,
    # with other expressions. You can pass more than one dictionaries.
    dicts=[emoticons]
)


C:\Users\user\anaconda3\lib\site-packages\ekphrasis\classes\tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))


Reading twitter - 1grams ...
Reading twitter - 2grams ...


C:\Users\user\anaconda3\lib\site-packages\ekphrasis\classes\exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in


Reading twitter - 1grams ...


## Membuat Fungsi

In [6]:
# panggil ekphrasis

def bersih_data(text):
    return " ".join(text_processor.pre_process_doc(text))
# fungsi dari AMS 01-03. silakan cek bagaimana saya merubah menjadi fungsi

def non_ascii(text):
    return text.encode('ascii', 'replace').decode('ascii')

def remove_space_alzami(text):
    return " ".join(text.split())

def remove_emoji_alzami(text):
    return ' '.join(re.sub("([x#][A-Za-z0-9]+)"," ", text).split())

def remove_tab(text):
    return text.replace('\\t'," ").replace('\\n'," ").replace('\\u'," ").replace('\\',"")

def remove_tab2(text):
    return re.sub('\s+',' ',text)

def remove_rt(text):
    return text.replace('RT'," ")

def remove_mention(text):
    return ' '.join(re.sub("([@#][A-Za-z0-9]+)|(\w+:\/\/\S+)"," ", text).split())

def remove_incomplete_url(text):
    return text.replace("http://", " ").replace("https://", " ")

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def change_stripe(text):
    return text.replace('-'," ")

def lower(text):
    return text.lower()

def remove_single_char(text):
    return re.sub(r"\b[a-zA-Z]\b", "", text)

def remove_excessive_dot(text):
    return text.replace('..'," ")

def lower(text):
    return text.lower()

def remove_whitespace_LT(text):
    return text.strip()

def remove_whitespace_multiple(text):
    return re.sub('\s+',' ',text)

def remove_punctuation(text):
    remove = string.punctuation
    remove = remove.replace("_", "") # don't remove hyphens
    pattern = r"[{}]".format(remove) # create the pattern
    return re.sub(pattern, "", text) 
# hapus untuk <>
def remove_number_eks(text):
    return text.replace('<number>'," ")

def remove_angka(text):
    return re.sub(r"\d+", "", text) 

def remove_URL_eks(text):
    return text.replace('URL'," ").replace('url'," ")

def space_punctuation(text):
    return re.sub('(?<! )(?=[.,!?()])|(?<=[.,!?()])(?! )', r' ', text)

## Memanggil Fungsi

In [7]:
i = 0
final_string = []
s = ""
for text in df['Tweets'].values:
    filteredSentence = []
    EachReviewText = ""
    proc = remove_rt(text)
    proc = lower(proc)
    proc = change_stripe(proc)
    proc = remove_emoji_alzami(proc)
    proc = remove_tab(proc)
    proc = remove_tab2(proc)
    proc = non_ascii(proc)
    proc = remove_incomplete_url(proc)
    proc = remove_excessive_dot(proc)
    proc = remove_whitespace_LT(proc)
    proc = remove_whitespace_multiple(proc)
    proc = remove_single_char(proc)
    proc = space_punctuation(proc)
    proc = remove_punctuation(proc)
    proc = remove_space_alzami(proc)
    proc = bersih_data(proc)
    proc = remove_number_eks(proc)
    proc = remove_angka(proc) 
    proc = remove_URL_eks(proc)
    EachReviewText = proc
    final_string.append(EachReviewText)

In [8]:
df["step01"] = final_string
df.head(10)

,Unnamed: 0,Tweets,step01
0,0,@laaaanis wkkwkw tengkiuuu,laaaanis wkkwkw tengkiuuu
1,1,"Kamu yang salah, kok kamu yang marah",kamu yang salah kok kamu yang marah
2,2,"Hari ini ke Lotte World Korea, pake seragam se...",hari ini ke lotte world korea pake seragam sek...
3,3,@jehianps @samsungID Thx ingfo,jehianps samsungid thx ingfo
4,4,RT @kamubukanakuu: PERKARA JAMBU JD PANJANG,kamubukanakuu perkara jambu jd panjang
5,5,RT @barkhohum: konspirasi,barkhohum konspirasi
6,6,RT @yxxxb_07: Masuk akal sih 🤣,_ masuk akal sih
7,7,RT @sminyeop: diluar nalar wkwkkw,sminyeop diluar nalar wkwkkw
8,8,Viral seorang anak ditanya √25 = …? Dan anak t...,viral seorang anak ditanya dan anak tersebut...
9,9,"Bentar lagi bakal trip ke Korea, gak sabar ket...",bentar lagi bakal trip ke korea gak sabar kete...


## Menghapus Data yang kosong

In [9]:
df.info()
df_hapus = df[~df['step01'].str.contains(" ")]
df_hapus.info()
df_hapus.head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  100 non-null    int64 
 1   Tweets      100 non-null    object
 2   step01      100 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.5+ KB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4 entries, 60 to 85
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4 non-null      int64 
 1   Tweets      4 non-null      object
 2   step01      4 non-null      object
dtypes: int64(1), object(2)
memory usage: 128.0+ bytes


,Unnamed: 0,Tweets,step01
60,60,Semangat!!,semangat
71,71,君，頑張ってください!,
72,72,RT @nikone_25: 安倍元総理がお亡くなりになりました。言葉がありません。犯人が憎...,nikone_
85,85,semangat!!!!,semangat


In [10]:
df_new = df[~df.isin(df_hapus)].dropna()
df_new.info()
df_new

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96 entries, 0 to 99
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  96 non-null     float64
 1   Tweets      96 non-null     object 
 2   step01      96 non-null     object 
dtypes: float64(1), object(2)
memory usage: 3.0+ KB


,Unnamed: 0,Tweets,step01
0,0.0,@laaaanis wkkwkw tengkiuuu,laaaanis wkkwkw tengkiuuu
1,1.0,"Kamu yang salah, kok kamu yang marah",kamu yang salah kok kamu yang marah
2,2.0,"Hari ini ke Lotte World Korea, pake seragam se...",hari ini ke lotte world korea pake seragam sek...
3,3.0,@jehianps @samsungID Thx ingfo,jehianps samsungid thx ingfo
4,4.0,RT @kamubukanakuu: PERKARA JAMBU JD PANJANG,kamubukanakuu perkara jambu jd panjang
...,...,...,...
95,95.0,Heran kenapa banyak orang suka ngomongin orang...,heran kenapa banyak orang suka ngomongin orang...
96,96.0,@ViktorAxelsen halo,viktora halo
97,97.0,viktor pasti takut lawan kucing gemuk,viktor pasti takut lawan kucing gemuk
98,98.0,Latian buat main lawan viktor axelsen WKWKWKWK...,latian buat main lawan viktor wkwkwkwk cofobgqpo


## Normalisasi Kata Slang

In [11]:
# token
import nltk
from nltk.tokenize import word_tokenize 

In [12]:
def word_tokenize_wrapper(text):
  return word_tokenize(text)

In [13]:
df_new['tokens'] = df['step01'].apply(word_tokenize_wrapper)

In [14]:
df_new.head(10)

,Unnamed: 0,Tweets,step01,tokens
0,0.0,@laaaanis wkkwkw tengkiuuu,laaaanis wkkwkw tengkiuuu,"[laaaanis, wkkwkw, tengkiuuu]"
1,1.0,"Kamu yang salah, kok kamu yang marah",kamu yang salah kok kamu yang marah,"[kamu, yang, salah, kok, kamu, yang, marah]"
2,2.0,"Hari ini ke Lotte World Korea, pake seragam se...",hari ini ke lotte world korea pake seragam sek...,"[hari, ini, ke, lotte, world, korea, pake, ser..."
3,3.0,@jehianps @samsungID Thx ingfo,jehianps samsungid thx ingfo,"[jehianps, samsungid, thx, ingfo]"
4,4.0,RT @kamubukanakuu: PERKARA JAMBU JD PANJANG,kamubukanakuu perkara jambu jd panjang,"[kamubukanakuu, perkara, jambu, jd, panjang]"
5,5.0,RT @barkhohum: konspirasi,barkhohum konspirasi,"[barkhohum, konspirasi]"
6,6.0,RT @yxxxb_07: Masuk akal sih 🤣,_ masuk akal sih,"[_, masuk, akal, sih]"
7,7.0,RT @sminyeop: diluar nalar wkwkkw,sminyeop diluar nalar wkwkkw,"[sminyeop, diluar, nalar, wkwkkw]"
8,8.0,Viral seorang anak ditanya √25 = …? Dan anak t...,viral seorang anak ditanya dan anak tersebut...,"[viral, seorang, anak, ditanya, dan, anak, ter..."
9,9.0,"Bentar lagi bakal trip ke Korea, gak sabar ket...",bentar lagi bakal trip ke korea gak sabar kete...,"[bentar, lagi, bakal, trip, ke, korea, gak, sa..."


## Clean Kamus

In [15]:
normalized_word = pd.read_csv('kamus_clean.csv')

In [16]:
normalized_word_dict = {}

for index, row in normalized_word.iterrows():
    if row[0] not in normalized_word_dict:
        normalized_word_dict[row[0]] = row[1] 

def normalized_term(document):
    return [normalized_word_dict[term] if term in normalized_word_dict else term for term in document]

In [17]:
df_new['final_tokens'] = df_new['tokens'].apply(normalized_term)

In [18]:
i=0
final_string_tokens = []
for text in df_new['final_tokens'].values:
    EachReviewText = ""
    EachReviewText = ' '.join(text)
    final_string_tokens.append(EachReviewText)

In [19]:
df_new["step02"] = final_string_tokens
df_new.head(10)

,Unnamed: 0,Tweets,step01,tokens,final_tokens,step02
0,0.0,@laaaanis wkkwkw tengkiuuu,laaaanis wkkwkw tengkiuuu,"[laaaanis, wkkwkw, tengkiuuu]","[laaaanis, wkkwkw, tengkiuuu]",laaaanis wkkwkw tengkiuuu
1,1.0,"Kamu yang salah, kok kamu yang marah",kamu yang salah kok kamu yang marah,"[kamu, yang, salah, kok, kamu, yang, marah]","[kamu, yang, salah, kok, kamu, yang, marah]",kamu yang salah kok kamu yang marah
2,2.0,"Hari ini ke Lotte World Korea, pake seragam se...",hari ini ke lotte world korea pake seragam sek...,"[hari, ini, ke, lotte, world, korea, pake, ser...","[hari, ini, ke, lotte, world, korea, pake, ser...",hari ini ke lotte world korea pake seragam sek...
3,3.0,@jehianps @samsungID Thx ingfo,jehianps samsungid thx ingfo,"[jehianps, samsungid, thx, ingfo]","[jehianps, samsungid, thx, ingfo]",jehianps samsungid thx ingfo
4,4.0,RT @kamubukanakuu: PERKARA JAMBU JD PANJANG,kamubukanakuu perkara jambu jd panjang,"[kamubukanakuu, perkara, jambu, jd, panjang]","[kamubukanakuu, perkara, jambu, jd, panjang]",kamubukanakuu perkara jambu jd panjang
5,5.0,RT @barkhohum: konspirasi,barkhohum konspirasi,"[barkhohum, konspirasi]","[barkhohum, konspirasi]",barkhohum konspirasi
6,6.0,RT @yxxxb_07: Masuk akal sih 🤣,_ masuk akal sih,"[_, masuk, akal, sih]","[_, masuk, akal, sih]",_ masuk akal sih
7,7.0,RT @sminyeop: diluar nalar wkwkkw,sminyeop diluar nalar wkwkkw,"[sminyeop, diluar, nalar, wkwkkw]","[sminyeop, diluar, nalar, wkwkkw]",sminyeop diluar nalar wkwkkw
8,8.0,Viral seorang anak ditanya √25 = …? Dan anak t...,viral seorang anak ditanya dan anak tersebut...,"[viral, seorang, anak, ditanya, dan, anak, ter...","[viral, seorang, anak, ditanya, dan, anak, ter...",viral seorang anak ditanya dan anak tersebut m...
9,9.0,"Bentar lagi bakal trip ke Korea, gak sabar ket...",bentar lagi bakal trip ke korea gak sabar kete...,"[bentar, lagi, bakal, trip, ke, korea, gak, sa...","[bentar, lagi, bakal, trip, ke, korea, gak, sa...",bentar lagi bakal trip ke korea gak sabar kete...


In [20]:
df_new.to_csv('Data Set Bersih.csv',sep=";")